In [1]:
!pip install tensorflow==2.0.0-beta1
import tensorflow as tf

  Using cached tensorflow-2.0.0b1-cp37-cp37m-win_amd64.whl (55.1 MB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


You should consider upgrading via the 'c:\users\hp\anaconda3\python.exe -m pip install --upgrade pip' command.
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\U

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random

In [ ]:
with open("./traffice-signs-data/train.p", mode='rb') as training_data:
    train = pickle.load(training_data)
with open("./traffice-signs-data/valid.p", mode='rb') as validation_data:
    valid = pickle.load(validation_data)
with open("./traffice-signs-data/train.p", mode='rb') as testing_data:
    test = pickle.load(testing_data)

In [ ]:
X_train , y_train = train['features'], train['labels']
X_valid , y_valid = valid['features'], valid['labels']
X_test , y_test = test['features'], test['labels']

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
## Task 3 IMage visulaization

In [ ]:
i = np.random.randint(1, len(X_train))
plt.imshow(X_train[i])
y_train[i]

In [ ]:
'''Define The dimnensions of the plot grid'''
W_grid=5
L_grid=5
# fig, axes = plt.subplots(L_grid,W_grid)
# subplot return the figure object and axes object
# we can use the axes object to plot specific fig at various locations

fig, axes = plt.subplots(L_grid,W_grid,figsize=(10,10)) 
axes = axes.ravel() # Flatten the 5x5 matrix into 25 array
n_training = len(X_train) #get the len of training set

In [ ]:
for i in np.arange(0,W_grid*L_grid):
    index = np.random.randint(0,n_training)
    
    axes[i].imshow(X_train[index])
    axes[i].set_title(y_train[index], fontsize = 15)
    axes[i].axis('off')
    
plt.subplots_adjust(hspace=0.4)

In [ ]:
## task 4 convert image to gray scale and perform normalization

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
X_train_gray = np.sum(X_train/3, axis=3, keepdims=True)
X_valid_gray = np.sum(X_valid/3, axis=3, keepdims=True)
X_test_gray = np.sum(X_test/3, axis=3, keepdims=True)

In [ ]:
X_train_gray.shape
# output = (34799,32,32,1)

In [ ]:
# Normalize data
X_train_gray_norm = (X_train_gray-128)/128

In [ ]:
X_train_gray_norm
# Output = normalize data (all values of images is between 0 to 1)

In [ ]:
## Task 6: Build deep CNN model

from tensorflow.keras import datasets, layeres,models

CNN = models.Sequential()

CNN.add(layers.Conv2D(6,(5,5),activation='relu', input_shape=(32,32,1)))
CNN.add(layers.AveragePooling2D())

CNN.add(layers.Dropout(0.2))

CNN.add(layers.Conv2D(16,(5,5),activation='relu'))
CNN.add(layers.AveragePooling2D())

CNN.add(layers.Flatten())

CNN.add(layers.Dense(120,activation='relu'))
CNN.add(layers.Dense(84,activation='relu'))
CNN.add(layers.Dense(43,activation='softmax'))

CNN.summary()

In [ ]:
## Task 7 Compile and train CNN model
CNN.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = CNN.fit(X_train_gray_norm,
                 y_train,
                 epochs =5,
                 verbose=1,
                 validation_data=(X_validation_gray_norm,y_validation))

In [ ]:
## Task 8

score  = CNN.evaluate(X_test_gray_norm, y_test)
print('Test Accuracy: {}'.format(score[1]))

In [ ]:
history.history.keys()

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss= history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(accuracy))

plt.plot(epochs, loss,'ro',label='Training loss')
plt.plot(epochs, val_loss,'r',label='Validation loss')
plt.title('Training and Validation loss')

In [ ]:
epochs = range(len(accuracy))

plt.plot(epochs, accuracy,'ro',label='Training accuracy')
plt.plot(epochs, val_accuracy,'r',label='Validation accuracy')
plt.title('Training and Validation accuracy')

In [ ]:
predicted_classes = CNN.predict_classes(X_test_gray_norm)

y_true = y_test

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, predicted_classes) 
plt.figure(figsize=(25,25))
sns.heatmap(cm,annot=True)

In [ ]:
L=5
W=5

fig, axes = plt.subplots(L,W, figsize=(12,12))
axes = axes.ravel()

for i in np.arange(0, L*W):
    axes[i].imshow(X_test[i])
    axes[i].set_title('Prediction = {}\n True = {}'.format(predicted_classes[i], y_true[i]))
    axes[i].axis('off')
    
plt.subplots_adjust(wspace=1)